# Conversion of Noise Power Reports and Data to XML

Written by AWS 12/2015
Data Provided By Dazhen

In [2]:
import os,re
# Create a file object that opens the current format
input_directory=r'F:\Share\CTL\Data For Playing\NoiseCal'
os.chdir(input_directory)
file_names=os.listdir(os.getcwd())
input_file_path=r'F:\Share\CTL\Data For Playing\NoiseCal\Raw_15Mar11_11.10-1.txt'

In [3]:
file_names

['Cal_15Mar11_14.25.txt',
 'Meas_15Mar11_11.10-1.txt',
 'Meas_15Mar11_12.52-2.txt',
 'Meas_15Mar11_14.25-3.txt',
 'Raw_15Mar11_11.10-1.txt',
 'Raw_15Mar11_12.52-2.txt',
 'Raw_15Mar11_14.25-3.txt']

In [4]:
input_file=open(input_file_path)
file_line_list=[]
for line in input_file:
    file_line_list.append(line)

In [5]:
file_line_list[0] # Clearly an erroneous entry, do I fix in parsing?

'Date of test: 2015 March 11 8.0 - 12.0 GHz\n'

In [6]:
file_line_list[90]

'Device temperature \n'

In [7]:
raw_header_list=['Date of test:','Directory:','File name:','Frequency list (GHz):','Customer:',
                 'Address:','DUT:','Operator:','Remarks:','Amb Std:','Cryo Std:','Chk Std:','DUT 2:']

In [8]:
for line in file_line_list:
    print line

Date of test: 2015 March 11 8.0 - 12.0 GHz

Directory:  N:\artifact\X1\x12zhz.137\Calibrations\2015 March 11 8.0 - 12.0 GHz

File name:  Raw_15Mar11_11.10-1.txt



Frequency list (GHz):  8.000	9.000	10.000	11.000	12.000



Customer: Northrup Grumman Space Tech

Address:  2477 Manhattan Beach Blvd

Redondo Beach, CA 90278



DUT:  x12zhz.137

Operator: DG

Remarks: DUT 811454 Cal #1



Amb Std: THTZ.010 on port 1

Cryo Std: PS2ZCZ.200 on port 2

Chk Std: x12zhz.137 on port 5

DUT 2: 811454 on port 4







++++++++++++++++++++++++++++++++++++

DUT 1: x12zhz.137 on port 5

++++++++++++++++++++++++++++++++++++





Cryo temp 

 84.32      84.62      84.91      85.18      85.43

 84.32      84.62      84.91      85.18      85.43

 84.32      84.62      84.91      85.18      85.43



Ambient temp 

296.34     296.32     296.29     296.27     296.24

296.34     296.31     296.29     296.26     296.24

296.33     296.31     296.28     296.26     296.23



Device power 

 8.957      6.936     

In [9]:
header_pattern=re.compile('(?P<header_name>^.*?):(?P<header_value>.*)')

In [10]:
matches=re.search(header_pattern,file_line_list[0])

In [11]:
matches.group(0)

'Date of test: 2015 March 11 8.0 - 12.0 GHz'

In [12]:
key=matches.group('header_name')

In [13]:
matches.group('header_value')

' 2015 March 11 8.0 - 12.0 GHz'

In [14]:
key.replace(' ','_')

'Date_of_test'

In [15]:
header_pattern=re.compile('(?P<header_name>^.*?):(?P<header_value>.*)')
header_dictionary={}
for i,line in enumerate(file_line_list):
    try:
        match_list=re.search(header_pattern,line)
        if match_list:
            key=match_list.group('header_name').replace(' ','_')
            key=key.translate(None,'()~!#$%^&*{}[]<>?;')
            key=key.strip()
            value=match_list.group('header_value')
            if re.match('Address',key):
                value=value+', '+file_line_list[i+1]
            header_dictionary[key]=value
    except:
        pass

In [88]:
header_dictionary

{'Address': '  2477 Manhattan Beach Blvd, Redondo Beach, CA 90278\n',
 'Amb_Std': ' THTZ.010 on port 1',
 'Chk_Std': ' x12zhz.137 on port 5',
 'Cryo_Std': ' PS2ZCZ.200 on port 2',
 'Customer': ' Northrup Grumman Space Tech',
 'DUT': '  x12zhz.137',
 'DUT_1': ' x12zhz.137 on port 5',
 'DUT_2': ' 811454 on port 4',
 'Date_of_test': ' 2015 March 11 8.0 - 12.0 GHz',
 'Directory': '  N:\\artifact\\X1\\x12zhz.137\\Calibrations\\2015 March 11 8.0 - 12.0 GHz',
 'File_name': '  Raw_15Mar11_11.10-1.txt',
 'Frequency_list_GHz': '  8.000\t9.000\t10.000\t11.000\t12.000',
 'Operator': ' DG',
 'Remarks': ' DUT 811454 Cal #1'}

In [16]:
test='(a test_string)'

In [17]:
test.translate(None,'()!^$%&#@')

'a test_string'

In [18]:
import numpy as np
table_list=['Cryo temp','Ambient temp','Device power','Device temperature',
'Ambient power','Cryo power']
table_dictionary={}
data_start=0
start_list=[]
for table in table_list:
    for i,line in enumerate(file_line_list):
        if re.match(table,line):
            data_start=i+1
            table_dictionary[table]={'data_start':data_start}
            start_list.append(data_start)

In [19]:
table_dictionary

{'Ambient power': {'data_start': 145},
 'Ambient temp': {'data_start': 32},
 'Cryo power': {'data_start': 199},
 'Cryo temp': {'data_start': 27},
 'Device power': {'data_start': 37},
 'Device temperature': {'data_start': 91}}

In [96]:
start_list

[27, 32, 37, 91, 145, 199]

In [20]:
start_list.sort()

In [21]:
stop_list=[]
for start in start_list:
    try: 
        stop=start_list[start_list.index(start)+1]-2
        stop_list.append(stop)
    except:
        stop=len(file_line_list)
        stop_list.append(stop)

In [101]:
stop_list

[30, 35, 89, 143, 197, 251]

In [22]:
np.loadtxt(file_line_list[27:30])

array([[ 84.32,  84.62,  84.91,  85.18,  85.43],
       [ 84.32,  84.62,  84.91,  85.18,  85.43],
       [ 84.32,  84.62,  84.91,  85.18,  85.43]])

In [60]:
table_list=['Cryo temp','Ambient temp','Device power','Device temperature',
'Ambient power','Cryo power']
table_dictionary={}
data_table_dictionary={}
data_start=0
start_list=[]
stop_list=[]
for table in table_list:
    for i,line in enumerate(file_line_list):
        if re.match(table,line):
            data_start=i+1
            table_dictionary[data_start]=table
            start_list.append(data_start)
start_list.sort()
for start in start_list:
    try: 
        stop=start_list[start_list.index(start)+1]-2
        stop_list.append(stop)
        data_table_dictionary[table_dictionary[start]]=np.loadtxt(file_line_list[start:stop])
    except:
        stop=len(file_line_list)
        stop_list.append(stop)
        data_table_dictionary[table_dictionary[start]]=np.loadtxt(file_line_list[start:stop])
print table_dictionary
print stop_list

{32: 'Ambient temp', 27: 'Cryo temp', 37: 'Device power', 199: 'Cryo power', 145: 'Ambient power', 91: 'Device temperature'}
[30, 35, 89, 143, 197, 251]


In [25]:
data_table_dictionary[table_list[1]]

array([[ 296.34,  296.32,  296.29,  296.27,  296.24],
       [ 296.34,  296.31,  296.29,  296.26,  296.24],
       [ 296.33,  296.31,  296.28,  296.26,  296.23]])

# Problem getting the data into format
There is an issue with this data. It should be (index,frequency,value) format. Index is row, Frequency is column

In [38]:
shape=data_table_dictionary[table_list[1]].shape

In [44]:
table_attributes=[table.replace(' ','_').strip().translate(None,'()~!#$%^&*{}[]<>?;') for table in table_list]

In [33]:
import io
text=io.StringIO(unicode(header_dictionary['Frequency_list_GHz']))
freq_list=np.loadtxt(text)

In [42]:
freq_list

tuple_list=[(i,freq_list[j],data_table_dictionary[table_list[1]][i,j]) for i in range(shape[0]) for j in range(shape[1])]

In [51]:
column_heads=['Index','Frequency',table_attributes[1]]
data=[]
for row in tuple_list:
    dictionary_data={column_heads[i]:item for i,item in enumerate(row) }
    data.append(dictionary_data)

In [52]:
data

[{'Ambient_temp': 296.33999999999997, 'Frequency': 8.0, 'Index': 0},
 {'Ambient_temp': 296.31999999999999, 'Frequency': 9.0, 'Index': 0},
 {'Ambient_temp': 296.29000000000002, 'Frequency': 10.0, 'Index': 0},
 {'Ambient_temp': 296.26999999999998, 'Frequency': 11.0, 'Index': 0},
 {'Ambient_temp': 296.24000000000001, 'Frequency': 12.0, 'Index': 0},
 {'Ambient_temp': 296.33999999999997, 'Frequency': 8.0, 'Index': 1},
 {'Ambient_temp': 296.31, 'Frequency': 9.0, 'Index': 1},
 {'Ambient_temp': 296.29000000000002, 'Frequency': 10.0, 'Index': 1},
 {'Ambient_temp': 296.25999999999999, 'Frequency': 11.0, 'Index': 1},
 {'Ambient_temp': 296.24000000000001, 'Frequency': 12.0, 'Index': 1},
 {'Ambient_temp': 296.32999999999998, 'Frequency': 8.0, 'Index': 2},
 {'Ambient_temp': 296.31, 'Frequency': 9.0, 'Index': 2},
 {'Ambient_temp': 296.27999999999997, 'Frequency': 10.0, 'Index': 2},
 {'Ambient_temp': 296.25999999999999, 'Frequency': 11.0, 'Index': 2},
 {'Ambient_temp': 296.23000000000002, 'Frequency':

In [54]:
#now a single data table looks like 
data_table={'Data_Description':{'Index':'The order it was measured','Frequency':'Frequency in GHZ',
                               'Ambient_temp':'Ambient temperature measured in Kelvin'},'Data':""}
data_table['Data']=data

import pyMeasure.Code.DataHandlers.Measurements as M
new_table=M.DataTable(**data_table)
new_table.save()
print new_table

<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?><Data_Table><Data_Description><Index>The order it was measured</Index><Frequency>Frequency in GHZ</Frequency><Ambient_temp>Ambient temperature measured in Kelvin</Ambient_temp></Data_Description><Data><Tuple Ambient_temp="296.34" Frequency="8.0" Index="0"/><Tuple Ambient_temp="296.32" Frequency="9.0" Index="0"/><Tuple Ambient_temp="296.29" Frequency="10.0" Index="0"/><Tuple Ambient_temp="296.27" Frequency="11.0" Index="0"/><Tuple Ambient_temp="296.24" Frequency="12.0" Index="0"/><Tuple Ambient_temp="296.34" Frequency="8.0" Index="1"/><Tuple Ambient_temp="296.31" Frequency="9.0" Index="1"/><Tuple Ambient_temp="296.29" Frequency="10.0" Index="1"/><Tuple Ambient_temp="296.26" Frequency="11.0" Index="1"/><Tuple Ambient_temp="296.24" Frequency="12.0" Index="1"/><Tuple Ambient_temp="296.33" Frequency="8.0" Index="2"/><Tuple Ambient_temp="296.31" Frequency="9.0" Index="2"/><Tuple Ambient_temp="296.28"

In [61]:
#Now lets iterate over all the tables
data_table_list=[]
for k,table in enumerate(table_attributes):
    column_heads=['Index','Frequency',table]
    data_table={'Data_Description':{'Index':'The order it was measured','Frequency':'Frequency in GHZ',
                               table:""},'Data':""}
    data=[]
    shape=data_table_dictionary[table_list[k]].shape
    tuple_list=[(i,freq_list[j],data_table_dictionary[table_list[k]][i,j]) for i in range(shape[0]) for j in range(shape[1])]
    for row in tuple_list:
        dictionary_data={column_heads[i]:item for i,item in enumerate(row) }
        data.append(dictionary_data)
    data_table['Data']=data
    new_table=M.DataTable(**data_table)
    new_table.save()
    data_table_list.append(new_table)

In [66]:
# Now lets make a single report by creating a series of tables 
print data_table_list[1]
processing_instructions='<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?>'
str(data_table_list[0]).replace(processing_instructions,'')

<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?><Data_Table><Data_Description><Index>The order it was measured</Index><Frequency>Frequency in GHZ</Frequency><Ambient_temp></Ambient_temp></Data_Description><Data><Tuple Ambient_temp="296.34" Frequency="8.0" Index="0"/><Tuple Ambient_temp="296.32" Frequency="9.0" Index="0"/><Tuple Ambient_temp="296.29" Frequency="10.0" Index="0"/><Tuple Ambient_temp="296.27" Frequency="11.0" Index="0"/><Tuple Ambient_temp="296.24" Frequency="12.0" Index="0"/><Tuple Ambient_temp="296.34" Frequency="8.0" Index="1"/><Tuple Ambient_temp="296.31" Frequency="9.0" Index="1"/><Tuple Ambient_temp="296.29" Frequency="10.0" Index="1"/><Tuple Ambient_temp="296.26" Frequency="11.0" Index="1"/><Tuple Ambient_temp="296.24" Frequency="12.0" Index="1"/><Tuple Ambient_temp="296.33" Frequency="8.0" Index="2"/><Tuple Ambient_temp="296.31" Frequency="9.0" Index="2"/><Tuple Ambient_temp="296.28" Frequency="10.0" Index="2"/><Tuple Am

'<Data_Table><Data_Description><Index>The order it was measured</Index><Frequency>Frequency in GHZ</Frequency><Cryo_temp></Cryo_temp></Data_Description><Data><Tuple Cryo_temp="84.32" Frequency="8.0" Index="0"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="0"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="0"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="0"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="0"/><Tuple Cryo_temp="84.32" Frequency="8.0" Index="1"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="1"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="1"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="1"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="1"/><Tuple Cryo_temp="84.32" Frequency="8.0" Index="2"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="2"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="2"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="2"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="2"/></Data></Data_Table>'

In [75]:
data_table_list[3].to_list('Index')

['0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4',
 '4',
 '4',
 '4',
 '5',
 '5',
 '5',
 '5',
 '5',
 '6',
 '6',
 '6',
 '6',
 '6',
 '7',
 '7',
 '7',
 '7',
 '7',
 '8',
 '8',
 '8',
 '8',
 '8',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '19',
 '20',
 '20',
 '20',
 '20',
 '20',
 '21',
 '21',
 '21',
 '21',
 '21',
 '22',
 '22',
 '22',
 '22',
 '22',
 '23',
 '23',
 '23',
 '23',
 '23',
 '24',
 '24',
 '24',
 '24',
 '24',
 '25',
 '25',
 '25',
 '25',
 '25',
 '26',
 '26',
 '26',
 '26',
 '26',
 '27',
 '27',
 '27',
 '27',
 '27',
 '28',
 '28',
 '28',
 '28',
 '28',
 '29',
 '29',
 '29',
 '29',
 '29',


In [89]:
# so we can put all of these into a single text file 
output_data=''
processing_instructions='<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?>'
for table in data_table_list:
    output_data=output_data+str(table).replace(processing_instructions,'')+'\n'

In [90]:
output_data

'<Data_Table><Data_Description><Index>The order it was measured</Index><Frequency>Frequency in GHZ</Frequency><Cryo_temp></Cryo_temp></Data_Description><Data><Tuple Cryo_temp="84.32" Frequency="8.0" Index="0"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="0"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="0"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="0"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="0"/><Tuple Cryo_temp="84.32" Frequency="8.0" Index="1"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="1"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="1"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="1"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="1"/><Tuple Cryo_temp="84.32" Frequency="8.0" Index="2"/><Tuple Cryo_temp="84.62" Frequency="9.0" Index="2"/><Tuple Cryo_temp="84.91" Frequency="10.0" Index="2"/><Tuple Cryo_temp="85.18" Frequency="11.0" Index="2"/><Tuple Cryo_temp="85.43" Frequency="12.0" Index="2"/></Data></Data_Table>\n<Data_Table><Data_Description><

In [91]:
processing_instructions='<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_REPORT_STYLE_2.XSL"?>'
output_data=processing_instructions+'<Report>'+output_data+'</Report>'

In [92]:

path=r'F:\Share\pyMeasure\Data\Measurements\Data_Table_120415_17.xml'       

file_out=open(path,'w')
file_out.write(output_data)
file_out.close()

In [97]:
processing_instructions='<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?>'
data_table={'Data_Description':header_dictionary,'Data':""}
new_table=M.DataTable(**data_table)
output_data=str(new_table).replace(processing_instructions,'')+'\n'

for table in data_table_list:
    output_data=output_data+str(table).replace(processing_instructions,'')+'\n'
    
    
processing_instructions='<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_REPORT_STYLE_3.XSL"?>'
output_data=processing_instructions+'<Report>'+output_data+'</Report>'

path=r'F:\Share\pyMeasure\Data\Measurements\Data_Table_120415_19.xml'       

file_out=open(path,'w')
file_out.write(output_data)
file_out.close()

# Right here it is clear that I need a report or other container that can accept DataTable Class Structures

Report.add_data_table(data_table) like this or something. It would also be nice to have a dictionary to xml writer. 
This in text is just $f: (key:value) -> "<key>value</key>" $

In [106]:
def dictionary_to_xml(dictionary=None,char_between='\n'):
    string_output=''
    for key,value in dictionary.iteritems():
        xml_open="<"+str(key)+">"
        xml_close="</"+str(key)+">"
        string_output=string_output+xml_open+str(value)+xml_close+char_between
        
    return string_output

In [114]:
print dictionary_to_xml(header_dictionary,'\n')

<Customer> Northrup Grumman Space Tech</Customer>
<Frequency_list_GHz>  8.000	9.000	10.000	11.000	12.000</Frequency_list_GHz>
<Chk_Std> x12zhz.137 on port 5</Chk_Std>
<Cryo_Std> PS2ZCZ.200 on port 2</Cryo_Std>
<Amb_Std> THTZ.010 on port 1</Amb_Std>
<File_name>  Raw_15Mar11_11.10-1.txt</File_name>
<Remarks> DUT 811454 Cal #1</Remarks>
<Operator> DG</Operator>
<DUT_1> x12zhz.137 on port 5</DUT_1>
<Address>  2477 Manhattan Beach Blvd, Redondo Beach, CA 90278
</Address>
<Directory>  N:\artifact\X1\x12zhz.137\Calibrations\2015 March 11 8.0 - 12.0 GHz</Directory>
<DUT>  x12zhz.137</DUT>
<DUT_2> 811454 on port 4</DUT_2>
<Date_of_test> 2015 March 11 8.0 - 12.0 GHz</Date_of_test>



In [122]:

test_list=[1,2,3]
test=M.DataTable(None,test_list)

In [123]:
print test

<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?><Data_Table><Data><Tuple X0="1" X1="2" X2="3"/></Data></Data_Table>


In [128]:
test=M.DataTable(None,data_table_dictionary[table_list[1]].tolist())

In [129]:
print test

<?xml version="1.0" ?><?xml-stylesheet type="text/xsl" href="DEFAULT_MEASUREMENT_STYLE.XSL"?><Data_Table><Data><Tuple X0="[296.34, 296.32, 296.29, 296.27, 296.24]" X1="[296.34, 296.31, 296.29, 296.26, 296.24]" X2="[296.33, 296.31, 296.28, 296.26, 296.23]"/></Data></Data_Table>
